In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas_profiling
import category_encoders as ce
import catboost
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
%matplotlib inline 
pd.options.display.max_columns = None

In [ ]:
X_train = pd.read_csv("train_values.csv")
y_train = pd.read_csv("train_labels.csv")
test = pd.read_csv('test_values.csv')
train = pd.merge(X_train, y_train, on='row_id')

In [ ]:
to_log = ["loan_amount", "applicant_income", "loan_income_ratio"]
          

to_drop = ['row_id', 'number_of_owner-occupied_units', 'population', 'number_of_1_to_4_family_units']

num_cols = ["minority_population_pct",
            "tract_to_msa_md_income_pct"
            ]

colss = ['applicant_income', 
       'ffiecmedian_family_income',  
       'loan_income_ratio']

def prepare_data(df):
    
    df['loan_income_ratio'] = ((df['loan_amount'] / df['applicant_income']) *100)
#     df['lenders_acceptance_rate'] = df.groupby('lender')['accepted'].mean()
    
    df[to_log] = df[to_log].applymap(math.log)
    
    df.drop(to_drop, axis=1, inplace=True)
    
    for col in colss:
        df[col].fillna(df[col].median(), inplace=True)
    
    for col in num_cols:
        df[col].fillna(df[col].mean(), inplace=True)
    
    
    return df

In [ ]:
train_set = prepare_data(train)

x= train_set.drop('accepted', axis=1)
y= train_set.accepted

ce_target = ce.TargetEncoder(cols = ["lender"], smoothing = 5, return_df = True)
train_set1 = ce_target.fit_transform(x, y)